# Fine-tune masked ruBERT-tiny2

Загружаем библиотеки

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModel
from datasets import Dataset
import evaluate

Загружаем отобранные предложения. Предложения отбираются в скрипте precorrect.py

In [2]:
with open('CorrectDN.json', encoding="utf-8") as f:
    sentences = pd.read_json(f, orient='records')
sentences.tail()

,text,label
2295,В Орловской области пресекли попытку атаки бес...,1
2296,СБУ пришла с обысками в Киево-Печерскую лавру....,1
2297,"В ЕС рассказали, когда передадут Киеву первый ...",1
2298,Полностью. На Западе пришли в шок от маневра Р...,1
2299,"Опрос показал, сколько россиян положительно оц...",1


Собираем dataset

In [3]:
raw_ds = Dataset.from_pandas(sentences, preserve_index=False)
raw_ds = raw_ds.class_encode_column('label')
raw_ds.features

Stringifying the column:   0%|          | 0/2300 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/2300 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['1'], id=None)}

Делим dataset на train и test (80:20)

In [4]:
raw_ds = raw_ds.train_test_split(test_size=0.2, shuffle=True)

In [5]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

# model_checkpoint = "cointegrated/rubert-tiny2"
# model_checkpoint = "DeepPavlov/rubert-base-cased"
model_checkpoint = "ai-forever/ruBert-base"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Max retries exceeded with url: /sberbank-ai/ruBert-base/6ab6521e029933cfd0731798544bf7d51876905752e760919c94e5eb4e37f851?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1708007983&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwODAwNzk4M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9zYmVyYmFuay1haS9ydUJlcnQtYmFzZS82YWI2NTIxZTAyOTkzM2NmZDA3MzE3OTg1NDRiZjdkNTE4NzY5MDU3NTJlNzYwOTE5Yzk0ZTVlYjRlMzdmODUxP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=R3h6Po0KT9m95unTf8VUdXbT5MUKSAWTJ6EV5HGHdF9ihBpWJsaXPr3y0QCjAZPXfFriONjTKkMCzMSq3d9QlgmxPH7vvF7vd6k3MjFm1-JCmkWI304Vc9a-Z2plEZnicW5e0F580k4nlIxCO5lJ1bgxFyhLQpsbgVwR4EmNqOSZsYjHBzuQtytE0lBKnNbnhy0O3n9Y2BayKoqwtTuwMWqSNiLmKKAMRZ-m

In [6]:
text = "После начала российской военной [MASK] на Украине западные страны усилили санкционное давление на Москву"

In [18]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
#print (inputs)
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
# print (mask_token_index)
mask_token_logits = token_logits[0, mask_token_index, :]

# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

word = "спецоперации"
iii = tokenizer(word, return_tensors="pt")
moscow_token = iii["input_ids"][0][1].item()
print(tokenizer.mask_token_id)
print (moscow_token)

probs = torch.nn.functional.softmax(mask_token_logits, dim=1)
#print (probs[0][29571])
#print (probs[0][19534])
print (probs[0][moscow_token])

top_5_prob = torch.topk(probs, 5, dim=1)
val = top_5_prob.values[0].tolist()
ind = top_5_prob.indices[0].tolist()
for i in range(len(val)):
    print (f"'>>> {tokenizer.decode([ind[i]])} ({ind[i]})  >>>>: {val[i]}")
print ("-----------------")



# for i in range(len(val_all)):
#     if ind[i] == 17627:
#         print (f"'>>> {tokenizer.decode([ind_all[i]])}  >>>>: {val_all[i]}")

#for token in top_5_tokens:
#    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}' {token}")

4
61407
tensor(7.1256e-07, grad_fn=<SelectBackward0>)
'>>> службы (10969)  >>>>: 0.10826659202575684
'>>> операции (11394)  >>>>: 0.07997777312994003
'>>> политики (17480)  >>>>: 0.07752140611410141
'>>> силы (11885)  >>>>: 0.06181306019425392
'>>> кампании (22938)  >>>>: 0.046161551028490067
-----------------


Tokenized dataset

In [19]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# Use batched=True to activate fast multithreading!
tokenized_datasets = raw_ds.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/837 [00:00<?, ? examples/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 837
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 210
    })
})

In [ ]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:5]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

Chank size (Parameter!!!)

In [20]:
chunk_size = 128

In [21]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [26]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)

print (lm_datasets["test"])
print (lm_datasets["test"]["input_ids"])
print (lm_datasets["test"]["token_type_ids"])
print (lm_datasets["test"]["attention_mask"])
print (lm_datasets["test"]["word_ids"])
print (lm_datasets["test"]["labels"])

Map:   0%|          | 0/837 [00:00<?, ? examples/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 366
})
[[2, 76456, 44404, 29804, 2514, 54267, 4544, 548, 41592, 733, 5241, 7214, 46547, 33607, 36991, 1874, 7368, 76456, 44404, 18, 6, 29735, 312, 329, 5624, 16, 3771, 2749, 17, 28179, 36124, 5918, 37530, 39297, 16, 25916, 10440, 32465, 28991, 320, 55614, 16, 5177, 6645, 9022, 43037, 16, 11514, 19389, 76347, 30324, 17, 34912, 38545, 5918, 6041, 10440, 314, 16949, 43125, 1154, 7036, 3131, 3618, 6, 16, 17, 30205, 7214, 47208, 4845, 18, 282, 5076, 3452, 42271, 735, 14622, 13884, 10999, 17094, 38089, 27610, 644, 1813, 16191, 314, 19634, 42703, 48834, 4845, 16, 1046, 17094, 548, 63383, 1082, 9632, 24181, 22818, 8623, 314, 41449, 860, 64507, 33122, 31673, 22193, 548, 39922, 6, 59220, 6, 1928, 548, 48946, 55187, 18, 282, 12773, 36880, 735, 57865, 6762, 9519, 1154, 30649, 548], [39065, 6, 78650, 24324, 6, 41145, 7373, 40673, 12, 78044, 866, 22824, 13, 26075, 6, 11491, 6, 2068, 16, 115

Процент маскирования = 15% (Parameter!!!)

In [27]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

whole word masking, процент маскирования = 15% (Parameter!!!)

In [28]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.15


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

Маскирование всего набора за раз

In [29]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [30]:
lm_datasets = lm_datasets.remove_columns(["word_ids"])
eval_dataset = lm_datasets["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=lm_datasets["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
        "masked_token_type_ids": "token_type_ids"
    }
)
eval_dataset
print (eval_dataset["input_ids"])
print (eval_dataset["attention_mask"])
print (eval_dataset["labels"])
print (eval_dataset["token_type_ids"])

Map:   0%|          | 0/366 [00:00<?, ? examples/s]

[[2, 76456, 44404, 29804, 2514, 54267, 4544, 548, 41592, 733, 5241, 7214, 4, 33607, 36991, 1874, 7368, 76456, 44404, 18474, 4, 29735, 312, 4, 5624, 16, 3771, 2749, 17, 28179, 4, 5918, 37530, 39297, 16, 25916, 10440, 32465, 28991, 320, 55614, 16, 4, 6645, 9022, 43037, 16, 4, 19389, 76347, 30324, 17, 34912, 4, 5918, 6041, 10440, 314, 16949, 43125, 1154, 7036, 4, 3618, 6, 37747, 17, 30205, 7214, 47208, 4845, 4, 282, 55829, 3452, 42271, 735, 14622, 13884, 4, 4, 38089, 27610, 644, 4, 16191, 314, 19634, 42703, 48834, 4845, 16, 1046, 17094, 4, 63383, 1082, 9632, 24181, 22818, 8623, 4, 41449, 860, 64507, 33122, 4, 22193, 548, 4, 6, 59220, 6, 1928, 548, 48946, 55187, 18, 282, 12773, 36880, 735, 57865, 6762, 9519, 1154, 30649, 548], [4, 6, 78650, 24324, 6, 41145, 7373, 40673, 4, 78044, 866, 22824, 13, 26075, 6, 11491, 4, 2068, 16, 1150, 8213, 16, 705, 34964, 2686, 4, 16, 16226, 6796, 41309, 48946, 4, 17094, 18, 36814, 17, 16808, 9906, 1874, 11327, 37602, 4, 2389, 30588, 16, 1046, 9519, 314, 2885

inputs_ids содержит маскированные данные (токен маски = 4), labels содержит токены под маской

Batch_size (Parameter!!!)

In [40]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 16
train_dataloader = DataLoader(
    lm_datasets["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
    # collate_fn=whole_word_masking_data_collator
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

LOSS function definition

In [51]:
import torch
import torch.nn as nn


i = 0
cross_entropy_loss = nn.CrossEntropyLoss()

for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)
        # print (tokenizer.decode([batch["labels"][0][7].item()]))
        # probs = nn.functional.softmax(outputs.logits[0][7], dim=-1)
        # print (probs[batch["labels"][0][7].item()])

        print ("Loss: ", outputs.loss)
        # print (outputs.logits.shape)
        # print (batch["labels"].shape)
        # ooo = torch.topk(outputs.logits, k=1, dim=2)
        # print (ooo.values.shape)
        # print (ooo.indices.shape)
        print (outputs.logits.view(-1, tokenizer.vocab_size).shape)
        print (batch["labels"].view(-1).shape)
        loss2 = cross_entropy_loss(outputs.logits.view(-1, tokenizer.vocab_size), batch["labels"].view(-1))
        print ("Loss2: ", loss2)
        print (tokenizer.vocab_size)        # cel = cross_entropy_loss(outputs.logits[0], batch["labels"])
        # print (cel)
        i += 1
        if i == 1:
            break

: 

Optimizer (Parameter!!!)

In [17]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [18]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [19]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [20]:
output_dir = 'fine-train'

In [ ]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)